In [16]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from scipy.fftpack import dct
import os

In [17]:
THRESHOLD = 0.05
ALPHA = 0.97
FRAME_SIZE = 0.025
FRAME_STRIDE = 0.01
SAMPLE_RATE = 16000
NUM_FILTERS = 26
MIN_FREQ = 0
N_FFT = 512
WINDOW_SIZE = 32768
NUM_CEPS = 13

In [18]:
def remove_silence(signal, threshold = THRESHOLD):
    signal[np.abs(signal) < threshold] = 0
    return signal

In [19]:
def pre_emphasis(signal, alpha = ALPHA):
    emphasized_signal = np.append(signal[0], signal[1:] - alpha * signal[:-1])
    return emphasized_signal


In [20]:
def framing(signal, frame_size = FRAME_SIZE, frame_stride = FRAME_STRIDE, sample_rate = SAMPLE_RATE):
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate
    signal_length = len(signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    padded_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((padded_signal_length - signal_length))
    pad_signal = np.append(signal, z) 

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy = False)]
    return frames

In [21]:
def apply_window(frames, window_func = np.hamming):
    frames *= window_func(frames.shape[1])
    return frames

In [22]:
def create_mel_filterbank(sample_rate, num_filters = NUM_FILTERS, min_freq = MIN_FREQ, max_freq = None, n_fft = N_FFT):
    if max_freq is None:
        max_freq = sample_rate // 2
    #  The formula to convert frequency in Hertz to Mel is given by: Mel=2595×log_10(1+𝑓/700)
    mel_min = 2595 * np.log10(1 + min_freq / 700)
    mel_max = 2595 * np.log10(1 + max_freq / 700)
    mel_points = np.linspace(mel_min, mel_max, num_filters + 2)
    hz_points = 700 * (10**(mel_points / 2595) - 1)
    bin_points = np.floor((n_fft + 1) * hz_points / sample_rate).astype(int)
    
    filterbank = np.zeros((num_filters, n_fft // 2 + 1))
    for i in range(1, num_filters + 1):
        filterbank[i - 1, bin_points[i - 1]:bin_points[i]] = (bin_points[i] - bin_points[i - 1]) / (hz_points[i] - hz_points[i - 1])
        filterbank[i - 1, bin_points[i]:bin_points[i + 1]] = (bin_points[i + 1] - bin_points[i]) / (hz_points[i + 1] - hz_points[i])
    return filterbank

In [23]:
def compute_mfcc(signal, sample_rate = SAMPLE_RATE, num_ceps = NUM_CEPS):
    frames = framing(signal, sample_rate = sample_rate)
    frames *= WINDOW_SIZE
    frames = apply_window(frames)
    magnitude_spectrum = np.abs(np.fft.rfft(frames, n = N_FFT))
    mel_filterbank = create_mel_filterbank(sample_rate, num_filters = NUM_FILTERS, n_fft = N_FFT)
    mel_spectrum = np.dot(magnitude_spectrum, mel_filterbank.T)
    log_mel_spectrum = np.log(mel_spectrum + 1e-10) 
    mfcc = dct(log_mel_spectrum, type = 2, axis = 1, norm = 'ortho')[:, 1 : (num_ceps + 1)]  
    return mfcc

In [24]:
hisb_60_warsh_transcript_df = pd.read_excel('warsh_transcript/warshData_v2-1.xlsx', skiprows = 1)
hisb_60_warsh_transcript_df.reset_index(drop = True, inplace = True)
hisb_60_warsh_transcript_df = hisb_60_warsh_transcript_df.iloc[:-1]
Al_Fatihah = hisb_60_warsh_transcript_df[hisb_60_warsh_transcript_df['sura_no'] == 1]
hisb_60_warsh_transcript_df = hisb_60_warsh_transcript_df[(hisb_60_warsh_transcript_df['sura_no'] >= 87) & (hisb_60_warsh_transcript_df['sura_no'] <= 114)]

hisb_60_and_Al_Fatihah_warsh_transcript_df = pd.concat([Al_Fatihah, hisb_60_warsh_transcript_df])

In [25]:
hisb_60_and_Al_Fatihah_warsh_transcript_df = hisb_60_and_Al_Fatihah_warsh_transcript_df[['sura_no',
                                            'sura_name_en',
                                            'sura_name_ar',
                                            'aya_no',
                                            'aya_text']]
hisb_60_and_Al_Fatihah_warsh_transcript_df.index = range(298)
hisb_60_and_Al_Fatihah_warsh_transcript_df

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ ١
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ ٢
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِۖ ٣
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُۖ ٤
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ ٥
...,...,...,...,...,...
293,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ ٢
294,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ ٣
295,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ ٤
296,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ ٥


In [26]:
directories = ["warsh_al_husray_wav_files"]

In [27]:
safa_wav = []
wavs = [safa_wav]
for i in range(len(directories)) :
    for subdir_name in os.listdir(directories[i]):
        subdir_path = os.path.join(directories[i], subdir_name)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)
                file_path = file_path.replace("\\", "/")
                wavs[i].append(file_path)

In [28]:
al_husary_recitor = {
    'recitor_en': "Al husary",
    'recitor_ar': "الحصري",
    'recitor_reciting_aya_wav_path': wavs[0],
}

al_husary_recitor_df = pd.DataFrame(al_husary_recitor, index = None)
al_husary_recitor_df

,recitor_en,recitor_ar,recitor_reciting_aya_wav_path
0,Al husary,الحصري,warsh_al_husray_wav_files/001/001.wav
1,Al husary,الحصري,warsh_al_husray_wav_files/001/002.wav
2,Al husary,الحصري,warsh_al_husray_wav_files/001/003.wav
3,Al husary,الحصري,warsh_al_husray_wav_files/001/004.wav
4,Al husary,الحصري,warsh_al_husray_wav_files/001/005.wav
...,...,...,...
293,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav
294,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav
295,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav
296,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav


In [29]:
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript = pd.concat([hisb_60_and_Al_Fatihah_warsh_transcript_df,
                                                                    al_husary_recitor_df], axis=1)
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ ١,Al husary,الحصري,warsh_al_husray_wav_files/001/001.wav
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ ٢,Al husary,الحصري,warsh_al_husray_wav_files/001/002.wav
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِۖ ٣,Al husary,الحصري,warsh_al_husray_wav_files/001/003.wav
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُۖ ٤,Al husary,الحصري,warsh_al_husray_wav_files/001/004.wav
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ ٥,Al husary,الحصري,warsh_al_husray_wav_files/001/005.wav
...,...,...,...,...,...,...,...,...
293,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ ٢,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav
294,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ ٣,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav
295,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ ٤,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav
296,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ ٥,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav


In [30]:
data = []
for index, row in al_husary_hisb_60_and_Al_fatihah_audio_with_transcript.iterrows():
    # Load audio file
    sample_rate, signal = wav.read(row['recitor_reciting_aya_wav_path'])
    # Remove silence
    signal = remove_silence(signal)
    # Apply pre-emphasis
    signal = pre_emphasis(signal)
    # Compute MFCCs
    mfcc = compute_mfcc(signal, sample_rate = sample_rate)
    data.append(mfcc.tolist())

mfcc_df = pd.DataFrame({'mfcc': data})
mfcc_df

,mfcc
0,"[[-2.4634406944524327, -1.5383123251761925, 1...."
1,"[[-3.3955018908303343, -2.3693992954246417, 1...."
2,"[[-3.188414896084072, -1.7357254070772927, 1.5..."
3,"[[-2.7001521612947235, -2.2294401188351918, 1...."
4,"[[-2.3898817628579256, -1.9240266972858175, 1...."
...,...
293,"[[-7.565721071800729, -0.8454428205762259, -1...."
294,"[[-7.86116052114519, 0.024208204195479283, -0...."
295,"[[-7.018722538548964, 0.04796886567999542, -0...."
296,"[[-6.597775486837384, -0.4972706513070791, -1...."


In [31]:
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC = pd.concat([al_husary_hisb_60_and_Al_fatihah_audio_with_transcript,
                                                                    mfcc_df], axis = 1)
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path,mfcc
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ ١,Al husary,الحصري,warsh_al_husray_wav_files/001/001.wav,"[[-2.4634406944524327, -1.5383123251761925, 1...."
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ ٢,Al husary,الحصري,warsh_al_husray_wav_files/001/002.wav,"[[-3.3955018908303343, -2.3693992954246417, 1...."
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِۖ ٣,Al husary,الحصري,warsh_al_husray_wav_files/001/003.wav,"[[-3.188414896084072, -1.7357254070772927, 1.5..."
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُۖ ٤,Al husary,الحصري,warsh_al_husray_wav_files/001/004.wav,"[[-2.7001521612947235, -2.2294401188351918, 1...."
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ ٥,Al husary,الحصري,warsh_al_husray_wav_files/001/005.wav,"[[-2.3898817628579256, -1.9240266972858175, 1...."
...,...,...,...,...,...,...,...,...,...
293,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ ٢,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav,"[[-7.565721071800729, -0.8454428205762259, -1...."
294,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ ٣,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav,"[[-7.86116052114519, 0.024208204195479283, -0...."
295,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ ٤,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav,"[[-7.018722538548964, 0.04796886567999542, -0...."
296,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ ٥,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav,"[[-6.597775486837384, -0.4972706513070791, -1...."


In [32]:
ahkam_indexing = pd.read_excel('ahkam_indexing/Ahkam in ayat hisb 60 + 001_v2.xlsx')

In [33]:
columns_to_keep = ["madd_6_Lazim", "madd_246", "madd_6", "madd_2",
                    "Ikhfaa", "Idgham", "tafkhim", "qalqala", "imala"]
new_df = ahkam_indexing[columns_to_keep]

In [34]:
new_df

,madd_6_Lazim,madd_246,madd_6,madd_2,Ikhfaa,Idgham,tafkhim,qalqala,imala
0,[-1],[-1],[36],[31],[-1],[-1],"[18, 19]",[-1],[-1]
1,[-1],[24],[-1],[11],[-1],"[3, 18]","[4, 5, 6, 19, 20, 21]",[-1],[-1]
2,[-1],[21],[-1],[-1],[-1],[17],[-1],[-1],[-1]
3,[-1],[37],[-1],[-1],[-1],[-1],[-1],[-1],[-1]
4,[-1],[36],[-1],[19],[-1],[13],"[14, 15, 16, 17, 18, 20, 21, 34, 35]",[-1],[-1]
...,...,...,...,...,...,...,...,...,...
293,[-1],[14],[-1],[-1],"[11, 12, 13]",[10],[-1],[-1],[-1]
294,[-1],[15],[-1],[4],"[12, 13, 14]",[11],[-1],[-1],[-1]
295,[-1],[35],[-1],[-1],"[2, 32, 33, 34]",[-1],"[30, 31]",[-1],[-1]
296,[-1],[38],[-1],[-1],"[35, 36, 37]",[34],"[23, 24]",[-1],[-1]


In [35]:
special_characters = ['١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩', '٠', 'ۖ', '۞', '۩']
for char in special_characters:
    al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'] = al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'].str.replace(char, '')
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'] = al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC['aya_text'].str.strip()

In [36]:
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path,mfcc
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ,Al husary,الحصري,warsh_al_husray_wav_files/001/001.wav,"[[-2.4634406944524327, -1.5383123251761925, 1...."
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ,Al husary,الحصري,warsh_al_husray_wav_files/001/002.wav,"[[-3.3955018908303343, -2.3693992954246417, 1...."
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِ,Al husary,الحصري,warsh_al_husray_wav_files/001/003.wav,"[[-3.188414896084072, -1.7357254070772927, 1.5..."
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,Al husary,الحصري,warsh_al_husray_wav_files/001/004.wav,"[[-2.7001521612947235, -2.2294401188351918, 1...."
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ,Al husary,الحصري,warsh_al_husray_wav_files/001/005.wav,"[[-2.3898817628579256, -1.9240266972858175, 1...."
...,...,...,...,...,...,...,...,...,...
293,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav,"[[-7.565721071800729, -0.8454428205762259, -1...."
294,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav,"[[-7.86116052114519, 0.024208204195479283, -0...."
295,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav,"[[-7.018722538548964, 0.04796886567999542, -0...."
296,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav,"[[-6.597775486837384, -0.4972706513070791, -1...."


In [37]:
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing = pd.concat([al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC, new_df], axis = 1)


In [38]:
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path,mfcc,madd_6_Lazim,madd_246,madd_6,madd_2,Ikhfaa,Idgham,tafkhim,qalqala,imala
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ,Al husary,الحصري,warsh_al_husray_wav_files/001/001.wav,"[[-2.4634406944524327, -1.5383123251761925, 1....",[-1],[-1],[36],[31],[-1],[-1],"[18, 19]",[-1],[-1]
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ,Al husary,الحصري,warsh_al_husray_wav_files/001/002.wav,"[[-3.3955018908303343, -2.3693992954246417, 1....",[-1],[24],[-1],[11],[-1],"[3, 18]","[4, 5, 6, 19, 20, 21]",[-1],[-1]
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِ,Al husary,الحصري,warsh_al_husray_wav_files/001/003.wav,"[[-3.188414896084072, -1.7357254070772927, 1.5...",[-1],[21],[-1],[-1],[-1],[17],[-1],[-1],[-1]
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,Al husary,الحصري,warsh_al_husray_wav_files/001/004.wav,"[[-2.7001521612947235, -2.2294401188351918, 1....",[-1],[37],[-1],[-1],[-1],[-1],[-1],[-1],[-1]
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ,Al husary,الحصري,warsh_al_husray_wav_files/001/005.wav,"[[-2.3898817628579256, -1.9240266972858175, 1....",[-1],[36],[-1],[19],[-1],[13],"[14, 15, 16, 17, 18, 20, 21, 34, 35]",[-1],[-1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav,"[[-7.565721071800729, -0.8454428205762259, -1....",[-1],[14],[-1],[-1],"[11, 12, 13]",[10],[-1],[-1],[-1]
294,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav,"[[-7.86116052114519, 0.024208204195479283, -0....",[-1],[15],[-1],[4],"[12, 13, 14]",[11],[-1],[-1],[-1]
295,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav,"[[-7.018722538548964, 0.04796886567999542, -0....",[-1],[35],[-1],[-1],"[2, 32, 33, 34]",[-1],"[30, 31]",[-1],[-1]
296,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav,"[[-6.597775486837384, -0.4972706513070791, -1....",[-1],[38],[-1],[-1],"[35, 36, 37]",[34],"[23, 24]",[-1],[-1]


In [39]:
al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing.to_csv('ahkam_indexing/al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing.csv', index = False)

In [45]:
quran = pd.read_csv('ahkam_indexing/hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing_v2.csv')
quran

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path,mfcc,madd_6_Lazim,madd_246,madd_6,madd_2,Ikhfaa,Idgham,tafkhim,qalqala,imala
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.491397213802345, 1.1010472775054547, -0.0...",[-1],[-1],[36],[31],[-1],[-1],"[18, 19]",[-1],[-1]
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.6060128460700507, 2.0262079519491634, 0.7...",[-1],[24],[-1],[11],[-1],"[3, 18]","[4, 5, 6, 19, 20, 21]",[-1],[-1]
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.963568909731852, 1.5978105678210048, 0.53...",[-1],[21],[-1],[-1],[-1],[17],[-1],[-1],[-1]
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-1.7085122141038942, 2.050192216986418, 0.83...",[-1],[37],[-1],[-1],[-1],[-1],[-1],[-1],[-1]
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.488483895713236, 2.168361353104937, 1.165...",[-1],[36],[-1],[19],[-1],[13],"[14, 15, 16, 17, 18, 20, 21, 34, 35]",[-1],[-1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ,Abdul Basit,عبد الباسط,warsh_Abdul_Basit_wav_files/114/002.wav,"[[0.0, 0.0, 0.0, 3.826852553242415e-15, 0.0, 0...",[-1],[14],[-1],[-1],"[11, 12, 13]",[10],[-1],[-1],[-1]
890,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ,Abdul Basit,عبد الباسط,warsh_Abdul_Basit_wav_files/114/003.wav,"[[0.0, 0.0, 0.0, 3.826852553242415e-15, 0.0, 0...",[-1],[15],[-1],[4],"[12, 13, 14]",[11],[-1],[-1],[-1]
891,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ,Abdul Basit,عبد الباسط,warsh_Abdul_Basit_wav_files/114/004.wav,"[[0.0, 0.0, 0.0, 3.826852553242415e-15, 0.0, 0...",[-1],[35],[-1],[-1],"[2, 32, 33, 34]",[-1],"[30, 31]",[-1],[-1]
892,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ,Abdul Basit,عبد الباسط,warsh_Abdul_Basit_wav_files/114/005.wav,"[[0.0, 0.0, 0.0, 3.826852553242415e-15, 0.0, 0...",[-1],[38],[-1],[-1],"[35, 36, 37]",[34],"[23, 24]",[-1],[-1]


In [47]:
hisb_60_and_Al_fatihah_for_all_recitors_audio_path_and_transcript  = pd.concat([quran,al_husary_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing], ignore_index=True)
hisb_60_and_Al_fatihah_for_all_recitors_audio_path_and_transcript 

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path,mfcc,madd_6_Lazim,madd_246,madd_6,madd_2,Ikhfaa,Idgham,tafkhim,qalqala,imala
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.491397213802345, 1.1010472775054547, -0.0...",[-1],[-1],[36],[31],[-1],[-1],"[18, 19]",[-1],[-1]
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.6060128460700507, 2.0262079519491634, 0.7...",[-1],[24],[-1],[11],[-1],"[3, 18]","[4, 5, 6, 19, 20, 21]",[-1],[-1]
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.963568909731852, 1.5978105678210048, 0.53...",[-1],[21],[-1],[-1],[-1],[17],[-1],[-1],[-1]
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-1.7085122141038942, 2.050192216986418, 0.83...",[-1],[37],[-1],[-1],[-1],[-1],[-1],[-1],[-1]
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.488483895713236, 2.168361353104937, 1.165...",[-1],[36],[-1],[19],[-1],[13],"[14, 15, 16, 17, 18, 20, 21, 34, 35]",[-1],[-1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav,"[[-7.565721071800729, -0.8454428205762259, -1....",[-1],[14],[-1],[-1],"[11, 12, 13]",[10],[-1],[-1],[-1]
1188,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav,"[[-7.86116052114519, 0.024208204195479283, -0....",[-1],[15],[-1],[4],"[12, 13, 14]",[11],[-1],[-1],[-1]
1189,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav,"[[-7.018722538548964, 0.04796886567999542, -0....",[-1],[35],[-1],[-1],"[2, 32, 33, 34]",[-1],"[30, 31]",[-1],[-1]
1190,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav,"[[-6.597775486837384, -0.4972706513070791, -1....",[-1],[38],[-1],[-1],"[35, 36, 37]",[34],"[23, 24]",[-1],[-1]


In [48]:
hisb_60_and_Al_fatihah_for_all_recitors_audio_path_and_transcript

,sura_no,sura_name_en,sura_name_ar,aya_no,aya_text,recitor_en,recitor_ar,recitor_reciting_aya_wav_path,mfcc,madd_6_Lazim,madd_246,madd_6,madd_2,Ikhfaa,Idgham,tafkhim,qalqala,imala
0,1.0,Al-Fātiḥah,الفَاتِحة,1.0,اِ۬لْحَمْدُ لِلهِ رَبِّ اِ۬لْعَٰلَمِينَ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.491397213802345, 1.1010472775054547, -0.0...",[-1],[-1],[36],[31],[-1],[-1],"[18, 19]",[-1],[-1]
1,1.0,Al-Fātiḥah,الفَاتِحة,2.0,اَ۬لرَّحْمَٰنِ اِ۬لرَّحِيمِ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.6060128460700507, 2.0262079519491634, 0.7...",[-1],[24],[-1],[11],[-1],"[3, 18]","[4, 5, 6, 19, 20, 21]",[-1],[-1]
2,1.0,Al-Fātiḥah,الفَاتِحة,3.0,مَلِكِ يَوْمِ اِ۬لدِّينِ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.963568909731852, 1.5978105678210048, 0.53...",[-1],[21],[-1],[-1],[-1],[17],[-1],[-1],[-1]
3,1.0,Al-Fātiḥah,الفَاتِحة,4.0,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-1.7085122141038942, 2.050192216986418, 0.83...",[-1],[37],[-1],[-1],[-1],[-1],[-1],[-1],[-1]
4,1.0,Al-Fātiḥah,الفَاتِحة,5.0,اُ۪هْدِنَا اَ۬لصِّرَٰطَ اَ۬لْمُسْتَقِيمَ,Ibrahim_Aldosary,إبراهيم الدوسري,warsh_ibrahim_aldosary_wav_files_madani_hisb_6...,"[[-2.488483895713236, 2.168361353104937, 1.165...",[-1],[36],[-1],[19],[-1],[13],"[14, 15, 16, 17, 18, 20, 21, 34, 35]",[-1],[-1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,114.0,An-Nās,النَّاس,2.0,مَلِكِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/002.wav,"[[-7.565721071800729, -0.8454428205762259, -1....",[-1],[14],[-1],[-1],"[11, 12, 13]",[10],[-1],[-1],[-1]
1188,114.0,An-Nās,النَّاس,3.0,إِلَٰهِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/003.wav,"[[-7.86116052114519, 0.024208204195479283, -0....",[-1],[15],[-1],[4],"[12, 13, 14]",[11],[-1],[-1],[-1]
1189,114.0,An-Nās,النَّاس,4.0,مِن شَرِّ اِ۬لْوَسْوَاسِ اِ۬لْخَنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/004.wav,"[[-7.018722538548964, 0.04796886567999542, -0....",[-1],[35],[-1],[-1],"[2, 32, 33, 34]",[-1],"[30, 31]",[-1],[-1]
1190,114.0,An-Nās,النَّاس,5.0,اِ۬لذِے يُوَسْوِسُ فِے صُدُورِ اِ۬لنَّاسِ,Al husary,الحصري,warsh_al_husray_wav_files/114/005.wav,"[[-6.597775486837384, -0.4972706513070791, -1....",[-1],[38],[-1],[-1],"[35, 36, 37]",[34],"[23, 24]",[-1],[-1]


In [49]:
hisb_60_and_Al_fatihah_for_all_recitors_audio_path_and_transcript.to_csv('ahkam_indexing/hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing_v3.csv', index = False)
